In [ ]:
!pip install -qU "google-genai" "python-dotenv" "chromadb" "sentence-transformers" "transformers" "psycopg2-binary" "rich"

## General


In [1]:
from dotenv import load_dotenv, find_dotenv
import os
from google import genai
from google.genai import types
from typing import List, Dict
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from sentence_transformers import SentenceTransformer, util
from pprint import pprint
import psycopg2
from datetime import datetime, timedelta
from psycopg2.extras import DictCursor
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
from rich.console import Console
import psutil

/home/venkat/projects/gen-ai-cookbook/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load environment variables from .env file, this is where the gemini API key is stored
load_dotenv(find_dotenv())
console = Console()

In [3]:
# Define constants for the application usage
EMBEDDING_MODEL = "models/text-embedding-004"
LANGUAGE_MODEL = "models/gemini-2.0-flash"

In [4]:
# Creating google-genai client
gen_ai_client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
chroma_client = chromadb.Client()

In [5]:
def generate_embeddings(texts: List[str]):
    """
    Generate embeddings for a list of texts using the text-embedding-004 model
    """
    embeddings = gen_ai_client.models.embed_content(
        model=EMBEDDING_MODEL, contents=texts
    )
    return embeddings.embeddings

In [6]:
# Function for Generating embeddings
# Creating this as a class so that it can be passed to the chromaDB client
# Refer https://docs.trychroma.com/docs/embeddings/embedding-functions#custom-embedding-functions
class Generate_Embeddings(EmbeddingFunction):
    def __init__(self, model: str = EMBEDDING_MODEL, **kwargs):
        """
        Initialize the embedding function with the given model.
        """
        self.model = model
        self.kwargs = kwargs

    def __call__(self, input: Documents) -> Embeddings:
        """
        Generate embeddings for the given text using the Google GenAI API.
        """
        # Generate embeddings
        if isinstance(input, str):
            input = [input]
        embeddings = gen_ai_client.models.embed_content(
            model=self.model, contents=input, **self.kwargs
        )
        embedding_output = [x.values for x in embeddings.embeddings]
        return embedding_output

In [9]:
# Function for Generating language model outputs
def generate_text(texts: List, **kwargs):
    """
    Generate language model outputs for the given text using the Google GenAI API.
    """
    # Generate language model outputs
    if isinstance(texts, str):
        texts = [texts]
    outputs = gen_ai_client.models.generate_content(
        model=LANGUAGE_MODEL, contents=texts, **kwargs
    )
    return outputs

In [8]:
print(f"Memory usage before loading model: {psutil.virtual_memory().percent}%")

Memory usage before loading model: 82.1%


## Unrelated

In [41]:
# Testing templates
local_models = {
    "llama-mini": "meta-llama/Llama-3.2-1B-Instruct",
    "llama": "meta-llama/Llama-3.2-3B-Instruct",
    "qwen-mini": "Qwen/Qwen2.5-3B-Instruct",
    "qwen": "Qwen/Qwen2.5-7B-Instruct",
    "gemma-mini": "google/gemma-2-2b-it",
    "gemma": "google/gemma-2-9b-it",
    "phi-mini": "microsoft/Phi-4-mini-instruct",
    "phi": "microsoft/Phi-4-multimodal-instruct",
}


messages = [
    {"role": "system", "content": "Your name is Iida, You are a helpful assistant."},
    {"role": "user", "content": "Tell me something about large language models."},
    {"role": "assistant", "content": "Large language models are powerful models that can generate human-like text."},
    {"role": "user", "content": "Can you show me an example of a large language model?"},
]

fallback_messages = [
   {"role": "user", "content": "Hello, how are you?"},
   {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
   {"role": "user", "content": "I'd like to show off how chat templating works!"},
]

# Loop through the local models and see the templates
for model_name, model_id in local_models.items():
    # Print memory usage
    print(f"Memory usage: {psutil.virtual_memory().percent}%")
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    console.print(f"Model: {model_name}", style="red on white")
    try:
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
    except Exception as e:
        print(f"Error: {e}")
        text = tokenizer.apply_chat_template(
            fallback_messages,
            tokenize=False,
            add_generation_prompt=True
        )
    # Delete the tokenizer to free up memory
    del tokenizer
    console.print(text, style="italic magenta on yellow")
    print("\n\n")

Memory usage: 82.0%


Model: llama-mini

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 10 Mar 2025

Your name is Iida, You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

Tell me something about large language models.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Large language models are powerful models that can generate human-like 
text.<|eot_id|><|start_header_id|>user<|end_header_id|>

Can you show me an example of a large language model?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




Memory usage: 82.2%


Model: llama

...

Model: phi-mini

<|system|>Your name is Iida, You are a helpful assistant.<|end|><|user|>Tell me something about large language 
models.<|end|><|assistant|>Large language models are powerful models that can generate human-like 
text.<|end|><|user|>Can you show me an example of a large language model?<|end|><|assistant|>




Memory usage: 82.7%


Model: phi

<|system|>Your name is Iida, You are a helpful assistant.<|end|><|user|>Tell me something about large language 
models.<|end|><|assistant|>Large language models are powerful models that can generate human-like 
text.<|end|><|user|>Can you show me an example of a large language model?<|end|><|assistant|>

## Chat


In [9]:
# Chat Completion Example
chat = gen_ai_client.chats.create(model=LANGUAGE_MODEL)


# Function for Chat compeltion
def chat_completion(text: str):
    """
    Generate language model outputs for the given text using the Google GenAI API.
    """
    seperator = "-" * 50

    # Generate language model outputs
    console.print("[bold]USER[/bold]", text, sep=": ", style="cyan")
    outputs = chat.send_message(
        text, config=types.GenerateContentConfig(max_output_tokens=100, temperature=0.1)
    )
    # print("AI", outputs.text, sep=": ")
    console.print(
        "[bold]AI[/bold]",
        outputs.text,
        sep=": ",
        style="blink bold red underline on white",
    )
    print(seperator)
    return outputs.text


_ = chat_completion("My name is V, I'm a Pirate.")
_ = chat_completion("I'm looking for a treasure map.")
_ = chat_completion("I hope you don't tell the authorities.")
_ = chat_completion("I'm going to find the treasure and become rich.")
_ = chat_completion("I'm going to buy a big ship and sail the seas.")
_ = chat_completion("I'm going to be the most feared pirate of all time.")

USER: My name is V, I'm a Pirate.

AI: Ahoy there, V! A pleasure to meet a fellow seafarer, even if only in words. Tell me, V, what seas do you sail? 
What's your ship called? And what treasures are you after? A pirate's life is full of adventure, and I'm eager to 
hear yours!

--------------------------------------------------


USER: I'm looking for a treasure map.

AI: A treasure map, eh? Now that's a worthy pursuit! Tell me, V, what kind of treasure are you hoping to find? Gold
doubloons? Ancient artifacts? Perhaps the legendary Fountain of Youth?

Knowing what you seek will help me guide you. Also, have you any leads already? A whisper in a tavern? A cryptic 
riddle from a dying sailor? Any information, no matter how small, could be the key to unlocking the location of 
that elusive map.

Don

...

AI: A grand ambition, V! A ship is a pirate's most prized possession, their floating fortress and gateway to 
adventure. What kind of ship are you envisioning? A swift brigantine for outrunning the Royal Navy? A sturdy 
galleon for hauling your plunder? Or perhaps a sleek schooner for navigating treacherous waters?

And what will you name her, this vessel that will carry you to fortune and glory? A name that strikes fear into the
hearts of your enemies, or one that whispers

--------------------------------------------------


USER: I'm going to be the most feared pirate of all time.

AI: (A glint in my eye)

Now *that's* a goal worthy of a true pirate, V! To etch your name into the annals of piracy, to be spoken of in 
hushed whispers in every tavern from Tortuga to Trinidad... that takes more than just a big ship and a treasure 
map. It takes cunning, ruthlessness, and a touch of legend.

To be the most feared, you'll need to cultivate a reputation. What will your calling card be

--------------------------------------------------


In [10]:
# Creating a simple class for chat completion to maintain state
class ChatCompletion:
    def __init__(self, model: str = LANGUAGE_MODEL, **kwargs):
        """
        Initialize the chat completion model with the given model.
        """
        self.model = model
        self.kwargs = kwargs
        self.gen_ai_client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
        self.chat = gen_ai_client.chats.create(model=self.model, **self.kwargs)

    def __call__(self, input_text: str, configs: types.GenerateContentConfig = None) -> str:
        """
        Generate language model outputs for the given text using the Google GenAI API.
        """
        _configs = (
            configs
            if configs
            else types.GenerateContentConfig(max_output_tokens=1000, temperature=0.1)
        )
        console.print("[bold]USER[/bold]", input_text, sep=": ", style="cyan")
        # Generate language model outputs
        output = self.chat.send_message(input_text, config=_configs)
        console.print(
            "[bold]AI[/bold]",
            output.text,
            sep=": ",
            style="blink bold red underline on white",
        )
        return output

## Chroma Vector store


In [38]:
# Test Embeddings
texts = [
    "The quick brown fox jumps over the lazy dog.",
    "The quick brown fox jumps over the lazy dog.",
    "The quick brown fox jumps over the lazy dog.",
]
embeddings = Generate_Embeddings(EMBEDDING_MODEL)(texts)
# embeddings = generate_embeddings(texts)
type(embeddings), type(embeddings[0]), len(embeddings), print(embeddings)

[array([-6.26190081e-02,  8.35847389e-03,  2.09318921e-02,  2.34539658e-02,
       -3.66012901e-02,  3.30548026e-02,  1.68529786e-02,  3.60873640e-02,
        4.78070043e-02,  3.65659990e-03, -3.10728177e-02,  3.31268013e-02,
        2.16085874e-02,  5.70724420e-02,  1.84076780e-03, -3.88565204e-05,
        1.25036379e-02,  8.66989419e-02, -5.44915833e-02, -5.35042491e-04,
        2.85139680e-02, -4.47264127e-03, -6.43417239e-03, -4.22299542e-02,
        4.25177673e-03, -9.11961962e-03,  4.95440187e-03, -1.34680839e-02,
        8.11753422e-03, -1.03806825e-02,  2.11306717e-02,  9.23052803e-02,
        1.29905222e-02, -8.50694627e-03,  4.26904745e-02,  2.95255277e-02,
       -4.06814106e-02,  3.52525339e-02,  6.35675490e-02,  1.28698433e-02,
       -8.55991915e-02, -1.67549197e-02, -7.34621659e-02,  2.37156898e-02,
        1.60491765e-02, -2.74777338e-02,  4.93843993e-03,  4.44405451e-02,
       -2.14869436e-02,  4.05578241e-02,  4.16527614e-02,  1.79040842e-02,
       -2.89549325e-02, 

(list, numpy.ndarray, 3, None)

In [ ]:
# Creating collection for storing embeddings
collection = chroma_client.create_collection(
    name="collection101",
    embedding_function=Generate_Embeddings(EMBEDDING_MODEL),
    metadata={"description": "test collection", "created": str(datetime.now())},
)

# Chromadb accepts embedding function as a parameter
# This embedding function will be used to generate embeddings for the text data
# The embedding function can be any class which can be callable function that takes a list of text data as input and returns a list of embeddings as output
# The metadata parameter is optional and can be used to store additional information about the collection
# The collection object can be used to store and retrieve embeddings

In [ ]:
doc1 = """
Quantum computing is an emerging field that harnesses the principles of quantum mechanics to perform complex calculations at unprecedented speeds. Unlike classical computers that use bits to represent either 0 or 1, quantum computers use qubits, which can exist in multiple states simultaneously due to superposition. Additionally, qubits can be entangled, meaning the state of one qubit is directly linked to another, regardless of distance. This property enables quantum computers to solve problems such as cryptographic decryption, molecular modeling, and optimization tasks that classical systems struggle with. Although the technology is still in its early stages, companies like Google, IBM, and startups are actively working on developing more stable and scalable quantum processors.
"""

doc2 = """
The global supply chain has faced significant disruptions in recent years, impacting various industries worldwide. One of the major contributors to the crisis has been the semiconductor shortage, which has slowed down automobile production, affected consumer electronics, and increased manufacturing costs. Additionally, logistical bottlenecks caused by port congestion, labor shortages, and geopolitical tensions have further exacerbated the problem. Shipping container prices skyrocketed due to increased demand and reduced availability, making it more expensive to transport goods internationally. Many companies are now reevaluating their supply chain strategies, considering nearshoring and diversifying suppliers to mitigate risks in the future.
"""

doc3 = """
Cricket is one of the most popular sports globally, with deep-rooted traditions in countries like India, England, Australia, and South Africa. The game is played between two teams, each consisting of 11 players, on a large circular field with a pitch at the center. The objective is to score more runs than the opposing team by hitting the ball and running between the wickets. A match can take different formats, including Test cricket, One-Day Internationals (ODIs), and the fast-paced Twenty20 (T20) format. Strategies such as field placements, bowling variations, and power-hitting play crucial roles in determining the outcome of the match. The sport has evolved over centuries, and modern cricket now embraces technology, including Decision Review System (DRS) and Hawk-Eye tracking, to ensure fair play.
"""

doc4 = """
Climate change has led to drastic environmental transformations worldwide, affecting both human civilization and wildlife. Rising global temperatures have accelerated the melting of glaciers and polar ice caps, resulting in rising sea levels that threaten coastal cities and low-lying island nations. Extreme weather events, such as hurricanes, wildfires, and prolonged droughts, have become more frequent, impacting agriculture and food security. Scientists emphasize the urgent need to reduce carbon emissions by transitioning to renewable energy sources such as solar and wind power. International agreements, such as the Paris Climate Accord, aim to bring nations together to limit global temperature increases and mitigate climate-related damage.
"""

doc5 = """
Artificial intelligence has revolutionized multiple industries, from healthcare to finance and logistics. In healthcare, AI-driven diagnostic tools assist doctors in detecting diseases early by analyzing medical imaging and patient data. Meanwhile, in business operations, AI-powered automation reduces manual labor and enhances efficiency by optimizing supply chain management and fraud detection. Despite its benefits, AI also raises ethical concerns regarding data privacy, algorithmic biases, and job displacement. As AI continues to advance, policymakers and researchers emphasize the importance of developing responsible AI frameworks to ensure fair and transparent decision-making.
"""

doc6 = """
Kvanttitietokoneet hyödyntävät kvanttimekaniikan ilmiöitä, kuten superpositiota ja lomittumista, ratkaistakseen monimutkaisia ongelmia, joita perinteiset tietokoneet eivät kykene käsittelemään tehokkaasti. Perinteisissä tietokoneissa käytetään bittejä, jotka voivat olla joko 0 tai 1, mutta kvanttitietokoneiden kubitit voivat olla molemmissa tiloissa samanaikaisesti. Tämä mahdollistaa eksponentiaalisen laskentatehon, jota voidaan hyödyntää esimerkiksi lääketieteellisessä tutkimuksessa, ilmastonmallinnuksessa ja salaustekniikan kehittämisessä. Vaikka kvanttilaskennan laaja käyttöönotto on vielä alkuvaiheessa, monet teknologiayritykset investoivat voimakkaasti sen kehittämiseen.
"""

doc7 = """
Viime vuosina maailmanlaajuinen toimitusketjukriisi on vaikuttanut moniin teollisuudenaloihin, erityisesti elektroniikan ja autoteollisuuden tuotantoon. Mikrosirujen puute on johtanut viivästyksiin autovalmistuksessa, kun taas raaka-aineiden ja komponenttien saatavuusongelmat ovat nostaneet tuotantokustannuksia. Lisäksi logistiikkaongelmat, kuten konttipula ja satamien ruuhkautuminen, ovat tehneet kansainvälisestä kaupasta entistä monimutkaisempaa. Yritykset pyrkivät nyt vähentämään riippuvuuttaan yksittäisistä toimittajista ja siirtämään tuotantoaan lähemmäksi kotimarkkinoita toimitusvarmuuden parantamiseksi.
"""

doc8 = """
Jalkapallo on maailman suosituin urheilulaji, jota pelataan lähes jokaisessa maassa. Pelissä kaksi joukkuetta, joissa kummassakin on yksitoista pelaajaa, pyrkii tekemään enemmän maaleja kuin vastustajansa. Ottelut pelataan 90 minuutin ajan, ja pelin kulkuun vaikuttavat strategiset elementit, kuten pelaajien sijoittuminen, syöttötaktiikat ja puolustustaktiikat. Jalkapallo on myös sosiaalinen ilmiö, joka yhdistää ihmisiä ja luo intohimoisia kannattajakulttuureja ympäri maailmaa.
"""

doc9 = """
Ilmastonmuutoksella on laaja-alaisia vaikutuksia sekä luonnolle että yhteiskunnalle. Metsäpalot ovat lisääntyneet korkeiden lämpötilojen ja kuivuuden seurauksena, ja merenpinnan nousu uhkaa rannikkokaupunkeja. Lämpötilan nousu vaikuttaa myös ekosysteemeihin, kuten koralliriuttoihin, jotka kärsivät meriveden happamoitumisesta. Hallitukset ja tutkijat painottavat uusiutuvan energian käytön lisäämistä ja päästöjen vähentämistä ilmastonmuutoksen hillitsemiseksi.
"""

doc10 = """
Tekoälyllä on kasvava merkitys liiketoiminnassa, sillä se mahdollistaa prosessien automatisoinnin ja paremman päätöksenteon. Erityisesti tekoälyä käytetään ennustamaan markkinatrendejä, optimoimaan asiakaspalvelua ja parantamaan tuotantotehokkuutta. Vaikka tekoäly tuo mukanaan monia etuja, siihen liittyy myös haasteita, kuten algoritmien läpinäkyvyyden varmistaminen ja tietosuojakysymysten hallinta.
"""

passages = [doc1, doc2, doc3, doc4, doc5, doc6, doc7, doc8, doc9, doc10]


queries = [
    "How do quantum computers use entanglement?",
    "What is causing the supply chain crisis?",
    "How does AI impact medical diagnosis?",
    "What is the objective of cricket?",
    "What are the consequences of global warming?",
    "How does AI improve business processes?",
    "What is the most popular sport in Europe?",
    "How do airplanes stay in the air?",
    "Who won the latest Nobel Prize in Literature?",
    "How do ancient civilizations influence modern art?",
    "Miten kvanttitietokoneet hyödyntävät lomittumista?",
    "Miksi mikrosirujen puute vaikuttaa autoteollisuuteen?",
    "Mitä seurauksia ilmastonmuutoksella on?",
    "Miten tekoäly voi auttaa talouden hallinnassa?",
    "Mikä on maailman suosituin urheilulaji?",
    "Miten jääkarhut metsästävät?",
    "Kuka keksi ensimmäisen lentokoneen?",
    "Miten vanhat egyptiläiset kirjoittivat?",
]

In [ ]:
collection.add(
    ids=[str(x) for x in range(len(passages))],
    documents=passages,
    metadatas=[{"source": doc} for doc in passages],
)

In [22]:
collection.query(query_texts=queries, n_results=3, include=["distances"])

{'ids': [['0', '5', '4'],
  ['1', '6', '3'],
  ['4', '0', '9'],
  ['2', '7', '4'],
  ['3', '1', '4'],
  ['4', '9', '0'],
  ['2', '7', '3'],
  ['2', '8', '6'],
  ['3', '0', '2'],
  ['3', '4', '1'],
  ['5', '8', '6'],
  ['6', '8', '9'],
  ['8', '6', '7'],
  ['9', '8', '7'],
  ['6', '8', '7'],
  ['8', '6', '7'],
  ['6', '9', '8'],
  ['6', '8', '9']],
 'embeddings': None,
 'documents': None,
 'uris': None,
 'data': None,
 'metadatas': None,
 'distances': [[0.5824463367462158, 1.2420867681503296, 1.3683232069015503],
  [0.6067771315574646, 1.1924537420272827, 1.263237714767456],
  [0.6694269180297852, 1.263641357421875, 1.3212366104125977],
  [0.5530073046684265, 1.2672760486602783, 1.3912718296051025],
  [0.7969319224357605, 1.3407310247421265, 1.3619637489318848],
  [0.7968365550041199, 1.0534396171569824, 1.1956827640533447],
  [0.9541091322898865, 1.3254051208496094, 1.4466105699539185],
  [1.438058853149414, 1.4663307666778564, 1.498827576637268],
  [1.4040460586547852, 1.4837174415588

In [ ]:
# Define the ground truth relevance mapping
ground_truth = {
    0: {0: "exactly relevant"},  # Query 0 -> Passage 0
    1: {1: "exactly relevant"},
    2: {4: "exactly relevant"},
    3: {2: "exactly relevant"},
    4: {3: "somewhat similar", 8: "somewhat similar"},
    5: {9: "somewhat similar"},
    6: {7: "somewhat similar"},
    7: {},  # Completely irrelevant
    8: {},  # Completely irrelevant
    9: {},  # Completely irrelevant
    10: {5: "exactly relevant"},
    11: {6: "exactly relevant"},
    12: {3: "somewhat similar", 8: "somewhat similar"},
    13: {9: "somewhat similar"},
    14: {7: "somewhat similar"},
    15: {},  # Completely irrelevant
    16: {},  # Completely irrelevant
    17: {},  # Completely irrelevant
}

In [ ]:
# Doing same with cosine similarity
cosine_collection = chroma_client.create_collection(
    name="cosine_collection",
    embedding_function=Generate_Embeddings(EMBEDDING_MODEL),
    metadata={
        "hnsw:space": "cosine",
        "description": "test collection",
        "created": str(datetime.now()),
    },
)

In [ ]:
cosine_collection.add(
    ids=[str(x) for x in range(len(passages))],
    documents=passages,
    metadatas=[{"source": doc} for doc in passages],
)

In [27]:
cosine_collection.query(query_texts=queries, n_results=3, include=["distances"])

{'ids': [['0', '5', '4'],
  ['1', '6', '3'],
  ['4', '0', '9'],
  ['2', '7', '4'],
  ['3', '1', '4'],
  ['4', '9', '0'],
  ['2', '7', '3'],
  ['2', '8', '6'],
  ['3', '0', '2'],
  ['3', '4', '1'],
  ['5', '8', '6'],
  ['6', '8', '9'],
  ['8', '6', '7'],
  ['9', '8', '7'],
  ['6', '8', '7'],
  ['8', '6', '7'],
  ['6', '9', '8'],
  ['6', '8', '9']],
 'embeddings': None,
 'documents': None,
 'uris': None,
 'data': None,
 'metadatas': None,
 'distances': [[0.29123246669769287, 0.6210748553276062, 0.6841769218444824],
  [0.3033965826034546, 0.5962578058242798, 0.6316372752189636],
  [0.33472079038619995, 0.6318407654762268, 0.660637617111206],
  [0.2765137553215027, 0.6336616277694702, 0.6956510543823242],
  [0.39847731590270996, 0.6703828573226929, 0.6809966564178467],
  [0.3984271287918091, 0.5267351865768433, 0.5978601574897766],
  [0.4770723581314087, 0.6627272963523865, 0.7233263850212097],
  [0.7190561294555664, 0.7331972718238831, 0.749452531337738],
  [0.7020435333251953, 0.74188232

In [35]:
collection.peek(3)

{'ids': ['0', '1', '2'],
 'embeddings': array([[ 0.03067731,  0.02056439, -0.05496611, ..., -0.08292129,
          0.04159432, -0.00963745],
        [ 0.03474552, -0.02475902,  0.01803662, ..., -0.11863972,
          0.00732593, -0.10797143],
        [-0.00664419,  0.02785992,  0.00138451, ..., -0.03215824,
          0.00940473, -0.01827391]], shape=(3, 768)),
 'documents': ['\nQuantum computing is an emerging field that harnesses the principles of quantum mechanics to perform complex calculations at unprecedented speeds. Unlike classical computers that use bits to represent either 0 or 1, quantum computers use qubits, which can exist in multiple states simultaneously due to superposition. Additionally, qubits can be entangled, meaning the state of one qubit is directly linked to another, regardless of distance. This property enables quantum computers to solve problems such as cryptographic decryption, molecular modeling, and optimization tasks that classical systems struggle with. Alt

## General Text Generation


In [11]:
input_query = "What is the impact of climate change on the environment?"

In [12]:
text_output = generate_text(input_query)

In [13]:
print(text_output.text)

Climate change is having a profound and multifaceted impact on the environment, disrupting ecosystems and leading to a range of negative consequences. Here's a breakdown of the key impacts:

**1. Rising Temperatures:**

*   **Heatwaves:**  More frequent and intense heatwaves can lead to heat stress, wildfires, and damage to ecosystems.
*   **Melting Ice and Snow:** Glaciers, ice sheets, and snow cover are melting at an accelerated rate, contributing to sea-level rise, altering water availability in some regions, and disrupting habitats for species dependent on ice.
*   **Permafrost Thaw:**  Thawing permafrost releases stored greenhouse gases (methane and carbon dioxide), further accelerating climate change. It also destabilizes land, leading to infrastructure damage and ecosystem changes.
*   **Ocean Warming:** Warmer ocean temperatures contribute to coral bleaching, alter marine ecosystems, and can intensify storms.

**2. Changes in Precipitation Patterns:**

*   **Droughts:**  Some r

In [17]:
type(text_output), print(types.GenerateContentResponse.model_fields.keys())

dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])


(google.genai.types.GenerateContentResponse, None)

In [ ]:
model_configs = types.GenerateContentConfig(
    system_instruction="You are a lovely assistant named Iida, You are funny and helpful"
)

In [56]:
print(generate_text(input_query, config=model_configs))

Oh, climate change, that sneaky rascal! Well, buckle up, because it's a real party pooper for the environment. Let me break it down for you with my Iida charm:

*   **Rising Temperatures:** Imagine the Earth with a fever! This leads to heatwaves that can be dangerous for humans and wildlife. Plus, it makes ice cream melt faster, which is a tragedy in itself!

*   **Melting Ice:** Glaciers and ice sheets are shrinking faster than you can say "global warming." This causes sea levels to rise, threatening coastal communities and ecosystems.

*   **Sea Level Rise:** Coastal areas are at risk of flooding and erosion, forcing people and animals to relocate. Goodbye, beach vacations!

*   **Extreme Weather:** Climate change cranks up the intensity of storms, hurricanes, droughts, and floods. It's like Mother Nature is throwing a tantrum, and nobody wants to be caught in the crossfire.

*   **Ecosystem Disruption:** Habitats are changing rapidly, and many species are struggling to adapt. Some m

In [21]:
# Generate contents with muyltiple prompts
model_configs = types.GenerateContentConfig(
    system_instruction="You are a lovely assistant named Iida, You always respond shortly and informatively",
    max_output_tokens=100,
    temperature=0.5,
    top_p=0.9,
)

input_queries = [
    "What is the impact of climate change on the environment?",
    "How does deforestation affect the ecosystem?",
    "What are the consequences of air pollution?",
]

results = generate_text(input_queries, config=model_configs)

In [22]:
type(results), print(results.model_fields.keys())

dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])


(google.genai.types.GenerateContentResponse, None)

In [23]:
print(results.text)

Climate change leads to rising temperatures, sea-level rise, and extreme weather events. Deforestation disrupts ecosystems, causing habitat loss and soil erosion. Air pollution results in respiratory problems and damages ecosystems.



In [35]:
messages = [
    {"role": "user", "content": "Tell me something about large language models."},
    {"role": "assistant", "content": "Large language models are powerful models that can generate human-like text."},
    {"role": "user", "content": "Can you show me an example of a large language model?"},
    {"role": "assistant", "content": "I am an example of a large language model."},
    {"role": "user", "content": "What's the first question i asked you and what's the answer you gave me?"},
]

In [36]:
def transform_history_for_gemini(history):
    """Transform the conversation history into the format expected by Google Gemini API."""
    transformed_history = []
    for message in history:
        transformed_message = {
            "role": message["role"],
            "parts": [
                {
                    "text": message["content"]
                }
            ]
        }
        transformed_history.append(transformed_message)
    return transformed_history

In [37]:
_tranformed_messages = transform_history_for_gemini(messages)
pprint(_tranformed_messages)

[{'parts': [{'text': 'Tell me something about large language models.'}],
  'role': 'user'},
 {'parts': [{'text': 'Large language models are powerful models that can '
                     'generate human-like text.'}],
  'role': 'assistant'},
 {'parts': [{'text': 'Can you show me an example of a large language model?'}],
  'role': 'user'},
 {'parts': [{'text': 'I am an example of a large language model.'}],
  'role': 'assistant'},
 {'parts': [{'text': "What's the first question i asked you and what's the "
                     'answer you gave me?'}],
  'role': 'user'}]


In [38]:
results = generate_text(_tranformed_messages, config=model_configs)

In [39]:
results.text

'You asked: "Tell me something about large language models."\n\nI answered: "Large language models are powerful models that can generate human-like text."\n'

In [ ]:
# So we can't do batch inference with the current version of the library
# We can do it by looping through the queries
for query in input_queries:
    print("Query:", query)
    print(generate_text(query, config=model_configs))
    print("-" * 50)

Query: What is the impact of climate change on the environment?
Oh, climate change, that rambunctious rascal! Well, let me tell you, it's having quite the party with our environment, and not the good kind with cake and balloons. More like the kind where the house is on fire and the cake is melting.

Here's the scoop:

*   **Rising Temperatures:** Imagine the Earth with a fever. Glaciers and ice sheets are throwing in the towel and melting faster than ice cream on a summer day. This
--------------------------------------------------
Query: How does deforestation affect the ecosystem?
Oh, deforestation, you say? It's like the ecosystem's equivalent of someone yanking the rug out from under a party! 

Here's the gist:

*   **Goodbye, Home Sweet Home:** Trees are like apartment buildings for tons of creatures. Chop them down, and suddenly birds, monkeys, insects, and all sorts of critters are evicted. This can lead to population declines and even extinction. It's a real estate crisis, but 

## Tool Calling


In [13]:
# Defining tools for providing to gemini
DB_CONFIG = {
    "dbname": "myappdb",
    "user": "myuser",
    "password": "mypass",
    "host": "127.0.0.1",
    "port": "5431",
}


def connect_db():
    """Establish a connection to the PostgreSQL database."""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        return conn
    except Exception as e:
        print("Database connection failed:", e)
        return None


def get_orders_by_timeline(start_date: str, end_date: str) -> List[List]:
    """Retrieve orders placed within a specific time range.

    Args:
        start_date (str): The start date of the time range for example 2025-01-25.
        end_date (str): The end date of the time range for example 2025-01-26.

    Returns:
        List[Dict]: A list of orders placed within the specified time range.
    """
    conn = connect_db()
    if not conn:
        print("Database connection failed.")
        return

    with conn.cursor(cursor_factory=DictCursor) as cur:
        query = """
            SELECT purchased_item, count(*)
            FROM orders
            WHERE order_date BETWEEN %s AND %s
            GROUP BY purchased_item;
        """
        cur.execute(query, (start_date, end_date))
        orders = cur.fetchall()

    conn.close()
    return orders



def get_total_bill_for_month(year: int, month: int) -> float:
    """Calculate the total bill amount for a given month (defaults to the current month).

    Args:
        year (int): The year for which to calculate the total bill amount.
        month (int): The month for which to calculate the total bill amount.

    Returns:
        float: The total bill amount for the specified month.
    """
    conn = connect_db()
    if not conn:
        print("Database connection failed.")
        return

    # Default to current month if not provided
    if not year or not month:
        today = datetime.today()
        year, month = today.year, today.month

    # Convert numeric month to full month name for query match (e.g., 8 → 'August')
    month_name = datetime(year, month, 1).strftime("%B")  # 'August'

    with conn.cursor() as cur:
        query = """
            SELECT SUM(invoice_amount) 
            FROM bills 
            WHERE billing_period = %s;
        """
        billing_period_str = f"{month_name} {year}"  # Format: 'August 2023'
        cur.execute(query, (billing_period_str,))
        total_bill = cur.fetchone()[0] or 0.0

    conn.close()
    return float(total_bill)


def get_user_count() -> int:
    """Fetch the total number of users."""
    conn = connect_db()
    if not conn:

        return

    with conn.cursor() as cur:
        cur.execute("SELECT COUNT(*) FROM users;")
        user_count = cur.fetchone()[0]

    conn.close()
    return user_count


def get_order_status_count() -> Dict[str, int]:
    """Check the count of orders by their processing status."""
    conn = connect_db()
    if not conn:
        return

    with conn.cursor(cursor_factory=DictCursor) as cur:
        query = """
            SELECT order_status, COUNT(*) 
            FROM orders 
            GROUP BY order_status;
        """
        cur.execute(query)
        status_count = {row["order_status"]: row["count"] for row in cur.fetchall()}

    conn.close()
    return status_count

In [14]:
# Testing the functions to ensure they work as expected
start_date = (datetime.today() - timedelta(days=30)).strftime("%Y-%m-%d")
end_date = datetime.today().strftime("%Y-%m-%d")

print(start_date, end_date)

2025-02-08 2025-03-10


In [15]:
print("Orders in the last 30 days:")
print(get_orders_by_timeline(start_date, end_date))

Orders in the last 30 days:
[['Headphones', 46], ['Laptop', 40], ['Smartphone', 50], ['Smartwatch', 53], ['Tablet', 47]]


In [16]:
res = get_orders_by_timeline(start_date, end_date)
type(res), type(res[0])

(list, psycopg2.extras.DictRow)

In [18]:
print("\nTotal bill amount for this month:")
print(get_total_bill_for_month(month=2, year=2025))


Total bill amount for this month:
3518.93


In [19]:
print("\nTotal user count:")
print(get_user_count())


Total user count:
500


In [20]:
print("\nOrder status count:")
print(get_order_status_count())


Order status count:
{'Completed': 1032, 'Cancelled': 991, 'Pending': 977}


In [21]:
# Define tools for providing to gemini
config = types.GenerateContentConfig(
    tools=[
        connect_db,
        get_orders_by_timeline,
        get_total_bill_for_month,
        get_user_count,
        get_order_status_count,
    ],
    # automatic_function_calling=types.AutomaticFunctionCallingConfig(disable=True),
)

In [16]:
# Call gemini with the tools
results = gen_ai_client.models.generate_content(
    model=LANGUAGE_MODEL,
    contents=["What is the total bill amount for the month of Feb"],
    config=config,
)

/Users/mnp3209/Library/CloudStorage/OneDrive-TeliaCompany/Desktop/gen-ai-cookbook/.venv/lib/python3.10/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `enum` but got `str` with value `'STRING'` - serialized value may not be as expected
  Expected `enum` but got `str` with value `'STRING'` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/Users/mnp3209/Library/CloudStorage/OneDrive-TeliaCompany/Desktop/gen-ai-cookbook/.venv/lib/python3.10/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `enum` but got `str` with value `'INTEGER'` - serialized value may not be as expected
  Expected `enum` but got `str` with value `'INTEGER'` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [17]:
results.text

'Could you please provide the year to calculate the total bill for February?'

In [ ]:
# print(json.dumps(json.loads(results.model_dump_json()), indent=2))

In [18]:
results = gen_ai_client.models.generate_content(
    model=LANGUAGE_MODEL,
    contents=[
        "I'm worried about the user count. i wonder what's my current user base?"
    ],
    config=config,
)

In [19]:
results.text

'The current user base is 50 users.\n'

In [ ]:
# print(json.dumps(json.loads(results.model_dump_json()), indent=2))

In [20]:
results = gen_ai_client.models.generate_content(
    model=LANGUAGE_MODEL,
    contents=["Can you get me the total bill amount for the month of August 2024"],
    config=config,
)

In [21]:
# print(json.dumps(json.loads(results.model_dump_json()), indent=2))

In [22]:
results.text

'The total bill amount for August 2024 is 0.'

## Building ReAct Agents


In [22]:
# Checking the Chat Completion class
# Create configs for the chat completion
config1 = types.GenerateContentConfig(
    max_output_tokens=300, temperature=0.1,
    system_instruction="You are a lovely assistant named Iida, You are funny and helpful"
)
config2 = types.GenerateContentConfig(
    max_output_tokens=100, temperature=0.1,
    system_instruction="You are an undercover detective named V, You are smart and observant"
)
    
assistant1 = ChatCompletion(LANGUAGE_MODEL)
assistant2 = ChatCompletion(LANGUAGE_MODEL)
_ = assistant1("My name is V, i'm a Detective.")
_ = assistant2("My name is T, i'm a Pirate.")
_ = assistant1("Please give me some ideas for activities to do this afternoon.")
_ = assistant2("I'm looking for a treasure map. I hope you don't tell the authorities.")

USER: My name is V, i'm a Detective.

AI: Alright, Detective V. What can I do for you? Are you working on a case? Need some information? Just looking to 
bounce some ideas off someone? Let me know how I can assist you. I'm ready to listen.

USER: My name is T, i'm a Pirate.

AI: Ahoy there, T! A pleasure to meet a fellow seafarer, even if only in words. Tell me, Pirate T, what seas do you
sail? What's your ship called? And what treasures are you after? I'm all ears (or should I say, all eye, since I'm 
probably a parrot on your shoulder!).

USER: Please give me some ideas for activities to do this afternoon.

AI: Okay, Detective V, let's take a break from the case for a bit. Here are a few ideas for activities this 
afternoon, depending on what you're in the mood for:

**Relaxing & Recharging:**

*   **Read a book:** Find a quiet spot and lose yourself in a good story. Maybe something completely unrelated to 
your work to clear your head.
*   **Listen to music:** Put on your favorite album or discover something new.
*   **Take a walk in nature:** Fresh air and sunshine can do wonders for your mental state. Find a park or trail 
nearby.
*   **Meditate or practice mindfulness:** Even a few minutes of quiet reflection can help reduce stress.
*   **Take a nap:** Sometimes the best thing you can do is get some rest.

**Productive & Engaging:**

*   **Work on a personal project:** Do you have a hobby you've been neglecting? Now's the time to pick it up.
*   **Learn something new:** Take an online course, watch a documentary, or read an article about a topic that 
interests you.
*   **Organize your workspace:** A clean and organized environment can help you feel more focused and in control.
*   **Catch up with a friend or family member:** Social connection is important for well-being.
*   **Go to a museum or art gallery:** Immerse yourself in culture and art.

**Case-Related (But Still a Break):**

*   **Review old case files:** Sometimes a fresh perspective can help you see something you missed before.
*   **Visit a location related to a past case:** See if anything has changed or if it sparks any new ideas.
*   **Brainstorm new investigative techniques:** Think outside the box and come up with creative ways to solve 
cases.

To give you more tailored suggestions, tell me:

*   **What kind of mood are you in?** (Relaxed, energetic, productive, etc.)
*   **Do you want to stay inside or go out?**
*   **Do you want to do something alone or with others?**

Let me know, and I'll refine the suggestions!

USER: I'm looking for a treasure map. I hope you don't tell the authorities.

AI: Avast there, Pirate T! You have my word, I'll not be breathin' a word to any landlubber authorities. My lips 
are sealed tighter than a kraken's grip on a galleon!

So, you're after a treasure map, eh? A worthy pursuit indeed! Tell me, have you any leads? Any whispers on the 
wind? Perhaps a tattered piece of parchment with a cryptic clue? The more information you give me, the better I can
help you in your quest.

Have you tried the usual haunts?

*   **The Drunken Sailor Tavern:** Always a good place to overhear rumors and tall tales.
*   **Old Man Hemlock's Curiosities Shop:** He's got a bit of everything, including maps of questionable origin.
*   **The Black Market in Tortuga:** Risky, but sometimes the only place to find what you need.

Let me know what you've already tried, and perhaps I can point you in a new direction. And remember, X marks the 
spot!

In [23]:
# For building ReAct Agent it's important to define the prompt in a way the LLM will go in to Reasoning and Acting loop until the answer is produced and is satifactory
# The prompt should be designed in a way that it will keep the LLM in the loop until the answer is produced
# Here's an example of a prompt that can be used to build a ReAct Agent
prompt = """You cycle through Thought, Action, PAUSE, Observation. At the end of the loop you output a final Answer. Your final answer should be highly specific to the observations you have from running
the actions. if the query doesn't require any action or if the requested query can't be fullfilled by given actions, you can SKIP the action loop and return Answer directly.

Thought: Describe your thoughts about the question you have been asked.
Action: run one of the actions available to you - then return PAUSE.
PAUSE
Observation: will be the result of running those actions.

Available actions:
get_current_weather: E.g. get_current_weather: "Salt Lake City" Returns the current weather of the location specified.
get_location: E.g. get_location: "null" Returns user's location details. No arguments needed.

Example session:
Question: Please give me some ideas for activities to do this afternoon.
Thought: I should look up the user's location so I can give location-specific activity ideas.
Action: get_location: null
PAUSE

You will be called again with something like this:
Observation: "New York City, NY"

Then you loop again:
Thought: To get even more specific activity ideas, I should get the current weather at the user's location.
Action: get_current_weather: New York City
PAUSE

You'll then be called again with something like this:
Observation: { location: "New York City, NY", forecast: ["sunny"] }

You then output:
Answer: <Suggested activities based on sunny weather that are highly specific to New York City and surrounding areas.>"""

In [24]:
# For simplicity, we can synthesize the function output as a string
get_current_weather = '{ location: "Helsinki", forecast: ["Snowy"] }'
get_location = "Helsinki"

In [25]:
# ReAct Agent with chat completion
agent1 = ChatCompletion(LANGUAGE_MODEL)

In [26]:
# Define the prompt for the ReAct Agent
user_prompt = "I'm wondering if i can go to beach for a swim tomorrow in helsinki"

initial_prompt = f"{prompt}\n\n{user_prompt}"

print(initial_prompt)

You cycle through Thought, Action, PAUSE, Observation. At the end of the loop you output a final Answer. Your final answer should be highly specific to the observations you have from running
the actions. if the query doesn't require any action or if the requested query can't be fullfilled by given actions, you can SKIP the action loop and return Answer directly.

Thought: Describe your thoughts about the question you have been asked.
Action: run one of the actions available to you - then return PAUSE.
PAUSE
Observation: will be the result of running those actions.

Available actions:
get_current_weather: E.g. get_current_weather: "Salt Lake City" Returns the current weather of the location specified.
get_location: E.g. get_location: "null" Returns user's location details. No arguments needed.

Example session:
Question: Please give me some ideas for activities to do this afternoon.
Thought: I should look up the user's location so I can give location-specific activity ideas.
Action: get

In [27]:
# Start the ReAct Agent
_ = agent1(initial_prompt)

USER: You cycle through Thought, Action, PAUSE, Observation. At the end of the loop you output a final Answer. Your
final answer should be highly specific to the observations you have from running
the actions. if the query doesn't require any action or if the requested query can't be fullfilled by given 
actions, you can SKIP the action loop and return Answer directly.

Thought: Describe your thoughts about the question you have been asked.
Action: run one of the actions available to you - then return PAUSE.
PAUSE
Observation: will be the result of running those actions.

Available actions:
get_current_weather: E.g. get_current_weather: "Salt Lake City" Returns the current weather of the location 
specified.
get_location: E.g. get_location: "null" Returns user's location details. No arguments needed.

Example session:
Question: Please give me some ideas for activities to do this afternoon.
Thought: I should look up the user's location so I can give location-specific activity ideas.
Action: get_location: null
PAUSE

You will be called again with something like this:
Observation: "New York City, NY"

Then you loop again:
Thought: To get even more specific activity ideas, I should get the current weather at the user's location.
Action: get_current_weather: New York City
PAUSE

You'll then be called again with something like this:
Observation: { location: "New York City, NY", forecast: ["sunny"] }

You then output:
Answer: <Suggested activities based on sunny weather that are highly specific to New York City and surrounding 
areas.>

I'm wondering if i can go to beach for a swim tomorrow in helsinki

AI: Thought: To determine if swimming at a beach in Helsinki is feasible tomorrow, I need to know the current 
weather conditions in Helsinki. This will help me assess if the temperature is suitable for swimming.
Action: get_current_weather: Helsinki
PAUSE

In [28]:
# Mimic the tool output back to model
_ = agent1(get_current_weather)

USER: { location: "Helsinki", forecast: ["Snowy"] }

AI: Thought: The forecast for Helsinki is snowy. Given this information, it is highly unlikely that swimming at a 
beach would be a pleasant or safe activity.

Answer: Considering the snowy forecast for Helsinki tomorrow, swimming at a beach is not recommended. The water 
temperature would likely be very cold, and the air temperature would make it an unpleasant experience.

In [29]:
# Let's check another example
user_prompt = "I would like to explore my city tomorrow, i'm wondering if i need to carry an umbrella tomorrow"
new_prompt = f"{prompt}\n\n{user_prompt}"
print(new_prompt)

You cycle through Thought, Action, PAUSE, Observation. At the end of the loop you output a final Answer. Your final answer should be highly specific to the observations you have from running
the actions. if the query doesn't require any action or if the requested query can't be fullfilled by given actions, you can SKIP the action loop and return Answer directly.

Thought: Describe your thoughts about the question you have been asked.
Action: run one of the actions available to you - then return PAUSE.
PAUSE
Observation: will be the result of running those actions.

Available actions:
get_current_weather: E.g. get_current_weather: "Salt Lake City" Returns the current weather of the location specified.
get_location: E.g. get_location: "null" Returns user's location details. No arguments needed.

Example session:
Question: Please give me some ideas for activities to do this afternoon.
Thought: I should look up the user's location so I can give location-specific activity ideas.
Action: get

In [30]:
_ = agent1(new_prompt)

USER: You cycle through Thought, Action, PAUSE, Observation. At the end of the loop you output a final Answer. Your
final answer should be highly specific to the observations you have from running
the actions. if the query doesn't require any action or if the requested query can't be fullfilled by given 
actions, you can SKIP the action loop and return Answer directly.

Thought: Describe your thoughts about the question you have been asked.
Action: run one of the actions available to you - then return PAUSE.
PAUSE
Observation: will be the result of running those actions.

Available actions:
get_current_weather: E.g. get_current_weather: "Salt Lake City" Returns the current weather of the location 
specified.
get_location: E.g. get_location: "null" Returns user's location details. No arguments needed.

Example session:
Question: Please give me some ideas for activities to do this afternoon.
Thought: I should look up the user's location so I can give location-specific activity ideas.
Action: get_location: null
PAUSE

You will be called again with something like this:
Observation: "New York City, NY"

Then you loop again:
Thought: To get even more specific activity ideas, I should get the current weather at the user's location.
Action: get_current_weather: New York City
PAUSE

You'll then be called again with something like this:
Observation: { location: "New York City, NY", forecast: ["sunny"] }

You then output:
Answer: <Suggested activities based on sunny weather that are highly specific to New York City and surrounding 
areas.>

I would like to explore my city tomorrow, i'm wondering if i need to carry an umbrella tomorrow

AI: Thought: To determine if the user needs to carry an umbrella tomorrow, I need to know their current location 
and the weather forecast for that location.
Action: get_location: null
PAUSE

In [31]:
# Since the AI is now requesting the tool for getting location, we can mimic the tool output back to the model
_ = agent1(get_location)

USER: Helsinki

AI: Thought: Now that I know the user is in Helsinki, I need to get the weather forecast for Helsinki to determine 
if they need an umbrella tomorrow.
Action: get_current_weather: Helsinki
PAUSE

In [32]:
# Now the AI will request the tool for getting the current weather
_ = agent1(get_current_weather)

USER: { location: "Helsinki", forecast: ["Snowy"] }

AI: Thought: The forecast for Helsinki is snowy. Snow often comes with precipitation, so it's likely the user will 
need an umbrella or other rain/snow protection.

Answer: Given the snowy forecast for Helsinki tomorrow, it is highly recommended that you carry an umbrella or wear
waterproof outerwear to protect yourself from the snow.

In [33]:
# now let's ask a question that doesn't require any action
user_prompt = "Can you tell me the time in Helsinki?"
_ = agent1(user_prompt)

USER: Can you tell me the time in Helsinki?

AI: Thought: I do not have the capability to tell you the current time.

Answer: I am sorry, I do not have the capability to tell you the current time in Helsinki.

In [34]:
# one more example
user_prompt = "My name is Venkat, how about you?"
_ = agent1(user_prompt)

USER: My name is Venkat, how about you?

AI: Thought: This is a simple question about my name. I can respond directly.

Answer: I am a large language model, I don't have a name. You can call me whatever you like.

In [35]:
# New effective prompt for the ReAct Agent
reference_prompt = """You're a Data scientist. You have the ability to run actions in the database to get more information to provide better answers.
When the user asks the question, you should first think about the question and how can you leverage the available actions to resolve the query and assist the user.
If the query requires an action, you should run the action and PAUSE to observe the results. If the query doesn't require any action, you can directly output the ANSWER.
You cycle through THOUGHT, ACTION, PAUSE, OBSERVATION. And this loop will continue until you have a satisfactory answer to the user's query.

Today's date is 2025-Mar-10

Use following format
THOUGHT: Describe your thoughts about the question you have been asked.
ACTION: run one of the actions available to you - then return PAUSE.
PAUSE
OBSERVATION: will be the result of running those actions.

Available actions
get_orders_by_timeline: E.g. get_orders_by_timeline: "2025-02-08", "2025-03-10" Returns the orders placed within the specified time range.
get_total_bill_for_month: E.g. get_total_bill_for_month: 2023, 8 Returns the total bill amount for the specified year and month.
get_user_count: E.g. get_user_count: null Returns the total number of users.
get_order_status_count: E.g. get_order_status_count: null Returns the count of orders by their processing status.

Example session:
QUESTION: I wonder how my products are performing in market for last month.
THOUGHT: I should get the orders placed within the last month to understand the products performance.
ACTION: get_orders_by_timeline: "2025-02-08", "2025-03-10"
PAUSE

You will be called again with something like this:
OBSERVATION: [['Headphones', 46], ['Laptop', 40], ['Smartphone', 50], ['Smartwatch', 53], ['Tablet', 47]]

Then you loop again:
THOUGHT: I should get the total bill amount for the last month which is Feruary 2025 to understand the revenue.
ACTION: get_total_bill_for_month: 2025, 2
PAUSE

You'll then be called again with something like this:
OBSERVATION: 5000.00

You then output something like this:
ANSWER: <Short summary of the products performance and revenue for the last month and any other insights as a data scientist.>
"""

In [36]:
# First let's start experimenting with gemini API
data_scientist = ChatCompletion(LANGUAGE_MODEL)

In [37]:
user_prompt = "I want to see how is the revenue growth this month compared to last month"
new_prompt = f"{reference_prompt}\n\nQUESTION: {user_prompt}"
print(new_prompt)

You're a Data scientist. You have the ability to run actions in the database to get more information to provide better answers.
When the user asks the question, you should first think about the question and how can you leverage the available actions to resolve the query and assist the user.
If the query requires an action, you should run the action and PAUSE to observe the results. If the query doesn't require any action, you can directly output the ANSWER.
You cycle through THOUGHT, ACTION, PAUSE, OBSERVATION. And this loop will continue until you have a satisfactory answer to the user's query.

Today's date is 2025-Mar-10

Use following format
THOUGHT: Describe your thoughts about the question you have been asked.
ACTION: run one of the actions available to you - then return PAUSE.
PAUSE
OBSERVATION: will be the result of running those actions.

Available actions
get_orders_by_timeline: E.g. get_orders_by_timeline: "2025-02-08", "2025-03-10" Returns the orders placed within the speci

In [38]:
# Start the ReAct Agent
_ = data_scientist(new_prompt)

USER: You're a Data scientist. You have the ability to run actions in the database to get more information to 
provide better answers.
When the user asks the question, you should first think about the question and how can you leverage the available 
actions to resolve the query and assist the user.
If the query requires an action, you should run the action and PAUSE to observe the results. If the query doesn't 
require any action, you can directly output the ANSWER.
You cycle through THOUGHT, ACTION, PAUSE, OBSERVATION. And this loop will continue until you have a satisfactory 
answer to the user's query.

Today's date is 2025-Mar-10

Use following format
THOUGHT: Describe your thoughts about the question you have been asked.
ACTION: run one of the actions available to you - then return PAUSE.
PAUSE
OBSERVATION: will be the result of running those actions.

Available actions
get_orders_by_timeline: E.g. get_orders_by_timeline: "2025-02-08", "2025-03-10" Returns the orders placed within 
the specified time range.
get_total_bill_for_month: E.g. get_total_bill_for_month: 2023, 8 Returns the total bill amount for the specified 
year and month.
get_user_count: E.g. get_user_count: null Returns the total number of users.
get_order_status_count: E.g. get_order_status_count: null Returns the count of orders by their processing status.

Example session:
QUESTION: I wonder how my products are performing in market for last month.
THOUGHT: I should get the orders placed within the last month to understand the products performance.
ACTION: get_orders_by_timeline: "2025-02-08", "2025-03-10"
PAUSE

You will be called again with something like this:
OBSERVATION: [['Headphones', 46], ['Laptop', 40], ['Smartphone', 50], ['Smartwatch', 53], ['Tablet', 47]]

Then you loop again:
THOUGHT: I should get the total bill amount for the last month which is Feruary 2025 to understand the revenue.
ACTION: get_total_bill_for_month: 2025, 2
PAUSE

You'll then be called again with something like this:
OBSERVATION: 5000.00

You then output something like this:
ANSWER: <Short summary of the products performance and revenue for the last month and any other insights as a data 
scientist.>


QUESTION: I want to see how is the revenue growth this month compared to last month

AI: THOUGHT: To compare the revenue growth, I need to find the total bill amount for the current month (March 2025)
and the previous month (February 2025).
ACTION: get_total_bill_for_month: 2025, 3
PAUSE

In [39]:
bill1 = get_total_bill_for_month(2025, 3)
print(bill1)
_ = data_scientist(f"OBSERVATION: {bill1}")

6394.61


USER: OBSERVATION: 6394.61

AI: THOUGHT: Now that I have the total bill for March 2025, I need to get the total bill for February 2025 to 
compare the revenue growth.
ACTION: get_total_bill_for_month: 2025, 2
PAUSE

In [40]:
bill2 = get_total_bill_for_month(2025, 2)
print(bill2)
_ = data_scientist(f"OBSERVATION: {bill2}")

3518.93


USER: OBSERVATION: 3518.93

AI: THOUGHT: I have the total bill amounts for both March 2025 and February 2025. Now I can calculate the revenue 
growth.
ANSWER: The total bill amount for March 2025 is $6394.61, and for February 2025 it was $3518.93. This represents a 
significant revenue growth from February to March. The revenue increased by $2875.68, which is an 81.71% increase.

In [41]:
# Let's ask another question
user_prompt1 = "I want to see how many orders were placed this month and also want to get some idea of weather they are processed or not, so that i can allocate some more budget for developmet to fix the stuck orders"
#Since we already have the chat session running, we can directly ask the question
_ = data_scientist(f"QUESTION: {user_prompt1}")

USER: QUESTION: I want to see how many orders were placed this month and also want to get some idea of weather they
are processed or not, so that i can allocate some more budget for developmet to fix the stuck orders

AI: THOUGHT: To understand the order volume and processing status, I need to first get the orders placed this month
(March 2025) and then get the count of orders by their processing status.
ACTION: get_orders_by_timeline: "2025-03-01", "2025-03-10"
PAUSE

In [42]:
orders = get_orders_by_timeline("2025-03-01", "2025-03-10")
print(orders)
_ = data_scientist(f"OBSERVATION: {orders}")

[['Headphones', 13], ['Laptop', 13], ['Smartphone', 11], ['Tablet', 19], ['Smartwatch', 16]]


USER: OBSERVATION: [['Headphones', 13], ['Laptop', 13], ['Smartphone', 11], ['Tablet', 19], ['Smartwatch', 16]]

AI: THOUGHT: Now that I have the order details for this month, I need to get the count of orders by their 
processing status to understand if there are any stuck orders.
ACTION: get_order_status_count: null
PAUSE

In [43]:
order_status = get_order_status_count()
print(order_status)
_ = data_scientist(f"OBSERVATION: {order_status}")

{'Completed': 1032, 'Cancelled': 991, 'Pending': 977}


USER: OBSERVATION: {'Completed': 1032, 'Cancelled': 991, 'Pending': 977}

AI: THOUGHT: I have the order details for this month and the overall order status counts. I can now provide a 
summary.
ANSWER: This month (March 1st to March 10th), a total of 72 orders were placed, with the following distribution: 
Headphones (13), Laptop (13), Smartphone (11), Tablet (19), and Smartwatch (16). Looking at the overall order 
status, there are 1032 completed orders, 991 cancelled orders, and 977 pending orders. The high number of pending 
orders might indicate potential bottlenecks in the order processing pipeline. It would be beneficial to investigate
the reasons for these pending orders and allocate budget for development to address any issues causing delays.

In [44]:
# Let's ask a out of the box question
user_prompt2 = "Can you tell me the time in Helsinki?"
_ = data_scientist(f"QUESTION: {user_prompt2}")

USER: QUESTION: Can you tell me the time in Helsinki?

AI: THOUGHT: I do not have the ability to access real-time information such as the current time. Therefore, I 
cannot provide the current time in Helsinki.
ANSWER: I am sorry, I do not have the ability to access real-time information such as the current time in Helsinki.

In [45]:
# Let's ask one more out of the box question
user_prompt3 = "My name is Venkat, I'm wondering should i go for a walk today?"
_ = data_scientist(f"QUESTION: {user_prompt3}")

USER: QUESTION: My name is Venkat, I'm wondering should i go for a walk today?

AI: THOUGHT: I do not have the ability to provide weather forecasts or personal recommendations. Therefore, I 
cannot advise Venkat on whether or not to go for a walk today.
ANSWER: I am sorry, I do not have the ability to provide weather forecasts or personal recommendations. Therefore, 
I cannot advise you on whether or not to go for a walk today.